<a href="https://colab.research.google.com/github/FrancescoTorella/progettoLabAI/blob/main/prova_progetto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#importo i moduli
import os
import numpy as np
!pip install rasterio
import rasterio
import torch
from torch.utils.data import Dataset,DataLoader,Subset
from torchvision import transforms
import albumentations as A
from sklearn.model_selection import train_test_split
import numpy as np
from PIL import Image



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.5/21.5 MB 34.8 MB/s eta 0:00:00


In [18]:
import random
from sklearn.model_selection import train_test_split

In [23]:
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
# Monta Google Drive
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [11]:
import os

# Percorso alla directory delle immagini
images_dir = '/content/drive/My Drive/progettoLabAi3/train/PS-RGBNIR'

# Lista i file nella directory
print(f"Contenuto della directory {images_dir}:")
print(os.listdir(images_dir))


Contenuto della directory /content/drive/My Drive/progettoLabAi3/train/PS-RGBNIR:
['SN6_Train_AOI_11_Rotterdam_PS-RGBNIR_20190823162315_20190823162606_tile_7879.tif', 'SN6_Train_AOI_11_Rotterdam_PS-RGBNIR_20190822155125_20190822155355_tile_7386.tif', 'SN6_Train_AOI_11_Rotterdam_PS-RGBNIR_20190823143529_20190823143813_tile_4438.tif', 'SN6_Train_AOI_11_Rotterdam_PS-RGBNIR_20190823130239_20190823130540_tile_3627.tif', 'SN6_Train_AOI_11_Rotterdam_PS-RGBNIR_20190822153558_20190822153835_tile_2542.tif', 'SN6_Train_AOI_11_Rotterdam_PS-RGBNIR_20190823073526_20190823073827_tile_7496.tif', 'SN6_Train_AOI_11_Rotterdam_PS-RGBNIR_20190822161837_20190822162134_tile_1591.tif', 'SN6_Train_AOI_11_Rotterdam_PS-RGBNIR_20190822072817_20190822073109_tile_10142.tif', 'SN6_Train_AOI_11_Rotterdam_PS-RGBNIR_20190822095638_20190822095937_tile_7442.tif', 'SN6_Train_AOI_11_Rotterdam_PS-RGBNIR_20190823153025_20190823153316_tile_5070.tif', 'SN6_Train_AOI_11_Rotterdam_PS-RGBNIR_20190804133705_20190804133940_tile_556

Nel file images-analyze avevo provato la seguente trasformazione per rendere le immagini menu scure, il risultato non sembrava meglio, chiederò più informazioni su quele trasformazione applicare e se è necessario

In [2]:

class ContrastStretching(object):
    def __init__(self, lower_percentile=2, upper_percentile=98):
        self.lower_percentile = lower_percentile
        self.upper_percentile = upper_percentile

    def __call__(self, img):
        """
        Args:
            img (PIL Image): Image to be transformed.

        Returns:
            PIL Image: Transformed image.
        """
        img_np = np.array(img)
        img_stretched = self.contrast_stretching(img_np)
        return Image.fromarray((img_stretched * 255).astype(np.uint8))

    def contrast_stretching(self, img):
        lower = np.percentile(img, self.lower_percentile)
        upper = np.percentile(img, self.upper_percentile)
        stretched_img = (img - lower) / (upper - lower)
        stretched_img = np.clip(stretched_img, 0, 1)
        return stretched_img

Questa è la funzione di splitting che salva i file in un txt come specificato nella mail

In [20]:


def split_dataset(image_dir, output_dir, train_ratio=0.7, val_ratio=0.15, test_ratio=0.15, seed=42):
    # Imposta il seed per la riproducibilità
    random.seed(seed)

    # Lista di tutte le immagini
    images = [f for f in os.listdir(image_dir) if f.endswith('.tif')]

    # Suddivisione train/val/test
    train_and_val, test = train_test_split(images, test_size=test_ratio, random_state=seed)
    train, val = train_test_split(train_and_val, test_size=val_ratio / (train_ratio + val_ratio), random_state=seed)

    # Salva i percorsi nei file .txt
    os.makedirs(output_dir, exist_ok=True)
    with open(os.path.join(output_dir, 'train.txt'), 'w') as f:
        for item in train:
            f.write("%s\n" % item)
    with open(os.path.join(output_dir, 'val.txt'), 'w') as f:
        for item in val:
            f.write("%s\n" % item)
    with open(os.path.join(output_dir, 'test.txt'), 'w') as f:
        for item in test:
            f.write("%s\n" % item)

# Esempio di utilizzo
image_dir = '/content/drive/My Drive/progettoLabAi3/train/PS-RGBNIR'
output_dir = '/content/drive/My Drive/progettoLabAi3/splits'
split_dataset(image_dir, output_dir)


Ho provato a definire la classe, questa è una base serviranno ovviamente degli adattamenti

In [22]:

class MultispectralDataset(Dataset):
    def __init__(self, images_dir, file_list, transform=None):
        """
        Args:
            images_dir (string): Directory with all the images.
            file_list (string): Path to the file containing the list of images.
            transform (callable, optional): Optional transform to be applied on a sample.
        """
        self.images_dir = images_dir
        self.transform = transform
        with open(file_list, 'r') as f:
            self.image_files = f.read().splitlines()

        # Debug: Stampa il numero di file trovati e i primi 5 nomi dei file
        # print(f"Found {len(self.image_files)} files in the list {file_list}")
        # print("First 5 files:", self.image_files[:5])

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = os.path.join(self.images_dir, self.image_files[idx])
        with rasterio.open(img_name) as src:
            img = src.read()


        R = img[0]
        G = img[1]
        B = img[2]
        NIR = img[3]


        R = self.normalize(R)
        G = self.normalize(G)
        B = self.normalize(B)
        NIR = self.normalize(NIR)


        img_combined = np.dstack((R, G, B, NIR)).astype(np.uint8)
        img_combined = Image.fromarray(img_combined)

        if self.transform:
            img_combined = self.transform(img_combined)

        return img_combined

    def normalize(self, channel):
        channel_min, channel_max = channel.min(), channel.max()
        return ((channel - channel_min) / (channel_max - channel_min) * 255).astype(np.uint8)

In [21]:

# Definisci la trasformazione per il dataset
transform = transforms.Compose([
    ContrastStretching(),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])


images_dir = '/content/drive/My Drive/progettoLabAi3/train/PS-RGBNIR'
multispectral_dataset = MultispectralDataset(images_dir=images_dir, transform=transform)


train_loader = DataLoader(multispectral_dataset, batch_size=32, shuffle=True, num_workers=4)



output_dir = '/content/drive/My Drive/progettoLabAi3/splits'
split_dataset(image_dir, output_dir)
